# Clean Data

In this notebook, we will be cleaning the scraped data. IF YOU HAVE NOT SCRAPED THE DATA, THEN THIS CODE WILL NOT WORK.

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter('ignore')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
# load scraped data
company_reviews = pd.read_csv('./company_reviews.csv')
company_reviews.head()

,company,rating,sub_work_life_balance,sub_culture_values,sub_diversity_inclusion,sub_career_opportunities,sub_compensation_benefits,sub_senior_management,author_info,employee_status,recommend,ceo_approval,outlook,pros,cons,advice_management
0,AMERICAN AIRLINES GROUP INC,5.0,NaN,NaN,NaN,NaN,NaN,NaN,"Mar 15, 2022 - Customer Relations",Current Employee,2,2,2,flexiblitiy is great amongest staff,the work load is overwhelming at times,0
1,AMERICAN AIRLINES GROUP INC,3.0,4.0,3.0,5.0,3.0,3.0,3.0,"Mar 15, 2022 - American Airlines Flight Attend...","Current Employee, more than 5 years",1,4,3,"Flexible schedules, great flight benefits, gre...","Constantly understaffed in all areas, Poor man...",Hire more people so we aren't constantly delay...
2,AMERICAN AIRLINES GROUP INC,5.0,5.0,5.0,5.0,5.0,5.0,5.0,"Mar 15, 2022 - Flight Attendant",Current Employee,1,1,1,Love my job . Love the freedom not being micro...,Reserve reserve reserve not being able to hold...,0
3,AMERICAN AIRLINES GROUP INC,1.0,NaN,NaN,NaN,NaN,NaN,NaN,"Mar 15, 2022 - Pilot",Former Employee,2,2,2,Quit after one month with the company.,"Working conditions, customer service, Terrance",Stop being a company full of uneducated low-li...
4,AMERICAN AIRLINES GROUP INC,3.0,4.0,2.0,5.0,3.0,3.0,3.0,"Mar 14, 2022 - Senior Project Manager in Fort ...","Former Employee, more than 10 years",2,1,1,Resources love to stay with this company for d...,Retention of resources Didn’t seem important I...,Offer retention bonuses for those loyal people...


In [3]:
'''clean columns'''

# extract information from 'author_info'
company_reviews['date'] = company_reviews['author_info'].str.split(' - ', n=1, expand=True)[[0]]
company_reviews['other_info'] = company_reviews['author_info'].str.split(' - ', n=1, expand=True)[[1]]
company_reviews['title'] = company_reviews['other_info'].str.split(' in ', n=1, expand=True)[[0]]
company_reviews['location'] = company_reviews['other_info'].str.split(' in ', n=1, expand=True)[[1]]
company_reviews['city'] = company_reviews['location'].str.split(', ', n=1, expand=True)[[0]]
company_reviews['state'] = company_reviews['location'].str.split(', ', n=1, expand=True)[[1]]

# extract information from 'employee status'
company_reviews['employee'] = company_reviews['employee_status'].str.split(', ', n=1, expand=True)[[0]]
company_reviews['years'] = company_reviews['employee_status'].str.split(', ', n=1, expand=True)[[1]]
company_reviews.loc[company_reviews.employee == 'Current Employee', 'current_employee'] = 1
company_reviews.loc[company_reviews.employee == 'Former Employee', 'current_employee'] = 0
company_reviews['years'] = company_reviews['years'].str.split(' ', expand=True)[[2]]

# drop columns
company_reviews.drop(['author_info', 'other_info', 'location', 'employee_status', 'employee'], inplace=True, axis=1)

In [4]:
# convert numbers used for symbols into words
for i,j in enumerate(['yes', np.nan, 'neutral', 'no']):
    company_reviews.loc[company_reviews.recommend == i+1, 'recommend'] = j
    company_reviews.loc[company_reviews.ceo_approval == i+1, 'ceo_approval'] = j
    company_reviews.loc[company_reviews.outlook == i+1, 'outlook'] = j

In [5]:
# combine pros, cons, and advice
company_reviews['advice_management'] = company_reviews['advice_management'].replace('0', '', regex=True)
company_reviews['advice_management'] = company_reviews['advice_management'].replace(np.nan, '', regex=True)
company_reviews['free_text_response'] = company_reviews['pros'] + ' ' + company_reviews['cons'] + ' ' + company_reviews['advice_management']
# company_reviews['free_text_response'] = company_reviews[['pros', 'cons', 'advice_management']].agg('. '.join, axis=1)
# company_reviews.drop(['pros', 'cons', 'advice_management'], inplace=True, axis=1)

In [6]:
# date from sting to date format
company_reviews['date'] = pd.to_datetime(company_reviews['date'])

In [7]:
company_reviews['year'] = company_reviews['date'].astype('str').str.split('-',expand=True)[0]
company_reviews['month'] = company_reviews['date'].astype('str').str.split('-',expand=True)[1]

In [8]:
company_reviews.head()

,company,rating,sub_work_life_balance,sub_culture_values,sub_diversity_inclusion,sub_career_opportunities,sub_compensation_benefits,sub_senior_management,recommend,ceo_approval,outlook,pros,cons,advice_management,date,title,city,state,years,current_employee,free_text_response,year,month
0,AMERICAN AIRLINES GROUP INC,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,flexiblitiy is great amongest staff,the work load is overwhelming at times,,2022-03-15,Customer Relations,None,None,None,1.0,flexiblitiy is great amongest staff the work l...,2022,03
1,AMERICAN AIRLINES GROUP INC,3.0,4.0,3.0,5.0,3.0,3.0,3.0,yes,no,neutral,"Flexible schedules, great flight benefits, gre...","Constantly understaffed in all areas, Poor man...",Hire more people so we aren't constantly delay...,2022-03-15,American Airlines Flight Attendant,New York,NY,5,1.0,"Flexible schedules, great flight benefits, gre...",2022,03
2,AMERICAN AIRLINES GROUP INC,5.0,5.0,5.0,5.0,5.0,5.0,5.0,yes,yes,yes,Love my job . Love the freedom not being micro...,Reserve reserve reserve not being able to hold...,,2022-03-15,Flight Attendant,None,None,None,1.0,Love my job . Love the freedom not being micro...,2022,03
3,AMERICAN AIRLINES GROUP INC,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quit after one month with the company.,"Working conditions, customer service, Terrance",Stop being a company full of uneducated low-li...,2022-03-15,Pilot,None,None,None,0.0,Quit after one month with the company. Working...,2022,03
4,AMERICAN AIRLINES GROUP INC,3.0,4.0,2.0,5.0,3.0,3.0,3.0,NaN,yes,yes,Resources love to stay with this company for d...,Retention of resources Didn’t seem important I...,Offer retention bonuses for those loyal people...,2022-03-14,Senior Project Manager,Fort Worth,TX,10,0.0,Resources love to stay with this company for d...,2022,03


In [9]:
# company_reviews.to_csv('C:/Users/19012/Documents/sp22-capstone/cleaned_reviews.csv', index=False)